<a href="https://colab.research.google.com/github/mateuszbarnacki/ML-Proj/blob/main/ThousandMinMax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from enum import Enum
from random import shuffle, randint
from math import floor
'''
Assumptions:  we have only 2 players
'''
'''
9 - 0
10 - 10
J - 2
D - 3
K - 4
A - 11
'''

class CardColor(Enum):
  PIK = 'PIK'
  TREFL = 'TREFL'
  KARO = 'KARO'
  KIER = 'KIER'

class CardValue(Enum):
  NINE = 0
  TEN = 10
  J = 2
  D = 3
  K = 4
  A = 11

class Card:
  def __init__(self, value, color):
    self.value = value
    self.color = color

  def show(self):
    print(self.value, ' ', self.color)

class Pack:
  def __init__(self):
    self.pack = self.createPack()
  
  def generatePack(self):
    pack = []
    for cardColor in CardColor:
      for cardvalue in CardValue:
        pack.append( Card(cardvalue.value, cardColor.value) )
    return pack
  
  def shufflePack(self, pack):
    return shuffle(pack)

  def createPack(self):
    pack = self.generatePack()
    self.shufflePack(pack)
    return pack
  
  def show(self):
    mappedPack = list(map(lambda card: ({
        'color': CardColor(card.color).name, 
        'value': CardValue(card.value).name 
        }),self.pack))
    for card in mappedPack:
      print(card['color'], ' ', card['value'])
  
  def getPack(self):
    return self.pack

class UserCards:
  def __init__(self, cards, dispatchesValues):
    self.cards = cards
    self.dispatchesValues = dispatchesValues
    self.estimatedCardsValue = self.estimateCardValues()

  def getCards(self):
    return self.cards
  
  def getCardsQuantity(self):
    return len(self.cards)

  def showCards(self):
    mappedPack = list(map(lambda card: ({
        'color': CardColor(card.color).name, 
        'value': CardValue(card.value).name 
        }),self.cards))
    for card in mappedPack:
      print(card['color'], ' ', card['value'])
  
  def addCards(self, cards):
    self.cards.extend(cards)
  
  def checkDispatches(self):
    kingsAndQueens = list(filter(lambda card: card.value ==  CardValue.D.value or card.value == CardValue.K.value, self.cards))
    possibleDispatches = {}
    for card in kingsAndQueens:
      if card.color in possibleDispatches:
        possibleDispatches[card.color].append(card.value)
      else:
        possibleDispatches[card.color] = [card.value]

    dispatches = []
    for color in possibleDispatches:
      if len(possibleDispatches[color]) == 2:
        dispatches.append(color)
    
    return dispatches

  def estimateCardValues(self):
    estimatedValue = 0
    dispatches = self.checkDispatches()
    
    for dispatch in dispatches:
      estimatedValue += self.dispatchesValues[dispatch]
    
    for card in self.cards:
      estimatedValue += card.value

    return floor(estimatedValue/10)*10

  def isWorthToTakePartInBidding(self, currentPrice):
    return self.estimatedCardsValue >= currentPrice

  def rejectWorthlessCards(self, quantityToReject):
    '''SHOULD BE SOME EXTRA MIN MAX LOGIC HERE'''
    '''
    HERE SHOULD BE IMPLEMENTED THE MINMAX TO DECIDE WHICH CARDS ARE THE BEST FOR PLAYER
    '''
    '''TEMPORARY'''
    while quantityToReject > 0:
      indexToRemove = randint(0, len(self.cards) - 1)
      self.cards.pop(indexToRemove)
      quantityToReject -= 1

  def handleExtraCards(self, extraCards):
    '''We have to assume that we add 4 cards to player. It is because we should not have cards that are not known in the game.'''
    self.addCards(extraCards)
    self.rejectWorthlessCards(4)

  def getValueToWin(self, minValueToWin):
    newEstimatedCardValues = self.estimateCardValues()

    if newEstimatedCardValues > minValueToWin:
      return newEstimatedCardValues
    return minValueToWin 

  def makeDecision(self, leadingGame, activeDispatch, leadingCard, opponentsCards):
    # bestCard, val = minimax(self.cards, opponentsCards, 3,leadingGame)
    # bestCard.show()
    # print(bestCard, val)
    '''MINMAX TO MAKING DECISION WHICH CARD SHOULD BE GIVEN'''
    return self.cards.pop(), False
    #return None, False

class Dispatch:
  def __init__(self, dispatchesValues):
    self.color = None
    self.isActive = False
    self.dispatchesValues = dispatchesValues
  
  def setDispatch(self, color):
    self.color = color
    self.isActive = True
  
  def getDispatchColor(self):
    if self.isActive:
      return self.color
    return None
  
  def getDispatchValue(self):
    return self.dispatchesValues[self.color]

class Game:
  def __init__(self):
    self.pack = Pack()
    self.valueToWin = 0
    self.users = []
    self.indexOfUserLeadingGame = -1
    self.usersPoints = [0, 0]
    self.iterations = -1
    self.dispatchesValues = {
        "PIK" : 40,
        "TREFL": 60,
        "KARO": 80,
        "KIER": 100
    }

  def playRound(self):
    self.prepareGame()
    return self.play()

  def play(self):
    currentCardsOnBoard = [None, None]
    activeDispatch = Dispatch(self.dispatchesValues)

    # while self.iterations > 0:
    currentCardsOnBoard[self.indexOfUserLeadingGame], isItDispatch = self.users[self.indexOfUserLeadingGame].makeDecision(True, activeDispatch, None, self.users[self.getIndexOfNextUser()].cards)
    currentCardsOnBoard[self.getIndexOfNextUser()], unusedVariable = self.users[self.getIndexOfNextUser()].makeDecision(False, activeDispatch, currentCardsOnBoard[self.indexOfUserLeadingGame], self.users[self.indexOfUserLeadingGame].cards)

    self.handleDispatch(isItDispatch, activeDispatch, currentCardsOnBoard)
    self.resolveCards(currentCardsOnBoard, activeDispatch)
    self.iterations -= 1

    return self.usersPoints

  def resolveCards(self, currentCardsOnBoard, activeDispatch):
    dispatchColor = activeDispatch.getDispatchColor()
    cardsValue = currentCardsOnBoard[self.indexOfUserLeadingGame].value + currentCardsOnBoard[self.getIndexOfNextUser()].value
    
    if dispatchColor:
      if currentCardsOnBoard[self.indexOfUserLeadingGame].color == dispatchColor and currentCardsOnBoard[self.getIndexOfNextUser()].color == dispatchColor:
        if currentCardsOnBoard[self.indexOfUserLeadingGame].value < currentCardsOnBoard[self.getIndexOfNextUser()].value:
          self.indexOfUserLeadingGame = self.getIndexOfNextUser()
      
        self.usersPoints[self.indexOfUserLeadingGame] += cardsValue

      else:
        if currentCardsOnBoard[self.indexOfUserLeadingGame].color == dispatchColor:
          self.usersPoints[self.indexOfUserLeadingGame] += cardsValue
        
        elif currentCardsOnBoard[self.getIndexOfNextUser()].color == dispatchColor:
          self.indexOfUserLeadingGame = self.getIndexOfNextUser()
          self.usersPoints[self.indexOfUserLeadingGame] += cardsValue
        
        else:
          if currentCardsOnBoard[self.indexOfUserLeadingGame].color == currentCardsOnBoard[self.getIndexOfNextUser()].color and currentCardsOnBoard[self.indexOfUserLeadingGame].value < currentCardsOnBoard[self.getIndexOfNextUser()].value:
            self.indexOfUserLeadingGame = self.getIndexOfNextUser()

          self.usersPoints[self.indexOfUserLeadingGame] += cardsValue

    else:
      if currentCardsOnBoard[self.indexOfUserLeadingGame].color == currentCardsOnBoard[self.getIndexOfNextUser()].color and currentCardsOnBoard[self.indexOfUserLeadingGame].value < currentCardsOnBoard[self.getIndexOfNextUser()].value:
        self.indexOfUserLeadingGame = self.getIndexOfNextUser()

      self.usersPoints[self.indexOfUserLeadingGame] += cardsValue



  def handleDispatch(self, isItDispatch, activeDispatch, currentCardsOnBoard):
    if isItDispatch:
        activeDispatch.setDispatch(currentCardsOnBoard[self.indexOfUserLeadingGame].color)
        self.usersPoints[self.indexOfUserLeadingGame] += activeDispatch.getDispatchValue()
        isItDispatch = False
    return isItDispatch

  def prepareGame(self):
    #Rozdanie
    users, hiddenCards = self.dealCards()
    
    #Licytacja
    indexOfUserLeadingGame, minValueToWin = self.bidding(users)

    #Zabranie karty przez wygranego licytacje
    users[indexOfUserLeadingGame].handleExtraCards(hiddenCards)

    #Podbicie stawki kart w licytacji
    minValueToWin = users[indexOfUserLeadingGame].getValueToWin(minValueToWin)

    self.valueToWin = minValueToWin
    self.users = users
    self.indexOfUserLeadingGame = indexOfUserLeadingGame
    self.iterations = users[indexOfUserLeadingGame].getCardsQuantity()

  def dealCards(self):
    cards = [[],[]]
    packList = self.pack.getPack()
    hiddenCards = packList[-4:]

    for cardIndex in range(len(packList) - 4):
      cards[cardIndex % 2].append(packList[cardIndex])

    usersCards = [ UserCards(cardsForUser, self.dispatchesValues) for cardsForUser in cards]
    
    return usersCards, hiddenCards

  def bidding(self, users):
    currentPrice = 100
    leadingUserIndex = 0

    while True:
      if users[(leadingUserIndex + 1) % 2].isWorthToTakePartInBidding(currentPrice):
        currentPrice += 10
        leadingUserIndex = (leadingUserIndex + 1) % 2
      else:
        break
    return leadingUserIndex, currentPrice
  
  def getIndexOfNextUser(self):
    if self.indexOfUserLeadingGame == 1:
      return 0
    return 1


usersPoints = Game().playRound()
usersPoints

[0, 13]

In [ ]:
from copy import deepcopy
'''
We have to make different decision depends on activeDispatch, 
giving card in the same color as card given by opponent (if he leads), 
if not, we have another case - how to decrease loss.
I dont know how to count the evaluation of the position #TO DO EVALUATION
'''
def minimax(usersCards, opponentsCards, chosenCard, depth, maximizingPlayer):
  #TO DO EVALUATION
  if depth == 0 or len(usersCards) == 1 and chosenCard != None:
    if maximizingPlayer:
      return usersCards[0], chosenCard.value + usersCards[0].value
    else:
      return usersCards[0], -(chosenCard.value + usersCards[0].value)

  bestCard = None

  if maximizingPlayer:
    maxEval = -1000
    for i in range(len(usersCards)):
      copiedCards = []
      for card in usersCards:
        copiedCards.append(deepcopy(card))
      chosenCard = copiedCards.pop(i)
      
      card, eval = minimax(opponentsCards, copiedCards, chosenCard, depth - 1, False)
    
      if eval > maxEval:
        maxEval = eval
        bestCard = card
    return (bestCard, maxEval)
  else: 
    minEval = 1000
    for i in range(len(usersCards)):
      copiedCards = []
      for card in usersCards:
        copiedCards.append(deepcopy(card))
      chosenCard = copiedCards.pop(i)
              
      card, eval = minimax(opponentsCards, copiedCards, chosenCard, depth - 1, True)

      if eval < minEval:
        minEval = eval
        bestCard = card
    return (bestCard, minEval)

In [ ]:
# def check_dispatch(current_card, cards):
#   is_dispatch = False
#   if current_card.value.name == CardValue.D.name:
#     color = current_card.color.name
#     for card in cards:
#       if card.color.name == color and card.value.name == CardValue.K.D:
#         is_dispatch = True
#         break
#   return is_dispatch

# def calculate_dispatch_value(color):
#   if color == "PIK":
#     return 40
#   elif color == "TREFL":
#     return 60
#   elif color == "KARO":
#     return 80
#   elif color == "KIER":
#     return 100
#   return 0

# def sec_minimax(user_cards, # A player cards 
#                 oponent_cards, # B player cards
#                 #user_last_card, # A player last card
#                 #oponent_last_card, # B player last card
#                 atut, # Required color (used to decrease number of options in decision tree)
#                 #user_score, # A user score (use to evaluate possible number of points)
#                 #depth, # depth of the decision tree (len(user_cards) + len(oponent_cards))
#                 maximizing_player): # flag
#   score = 0
#   if len(user_cards) == 1:
#     card = oponent_cards.pop()
#     return card, card.value.value

#   if len(oponent_cards) == 1:
#     card = user_cards.pop()
#     return card, card.value.value

#   if maximizing_player:
#     max_score = float("-inf")
#     # Try to decrease the number of options in decision tree by choosing the cards which are required by atut (advantage)
#     possible_next_cards = []
#     for card in user_cards:
#       if card.color.name == atut:
#         possible_next_cards.append(card)
    
#     if len(possible_next_cards) > 0:
#       for card in possible_next_cards:
#         new_list = user_cards.copy()
#         new_list.remove(card)
#         eval = sec_minimax(new_list, oponent_cards, atut, False)
#         # Calculate score with possible dispatch
#         if check_dispatch(card, new_list):
#           score = eval[1] + calculate_dispatch_value(card.color.name)
#         score = eval[1]
#         max_score = max(max_score, score)
#         if max_score == score:
#           best = eval
#     else:   
#       for card in user_cards:
#         new_list = user_cards.copy()
#         new_list.remove(card)
#         eval = sec_minimax(new_list, oponent_cards, atut, False)
#         # Calculate score with possible dispatch
#         if check_dispatch(card, new_list):
#           score = eval[1] + calculate_dispatch_value(card.color.name)
#         score = eval[1]
#         max_score = max(max_score, score)
#         if max_score == score:
#           best = eval
    
#     user_cards.remove(best[0])
#     if check_dispatch(best[0], user_cards):
#       atut = best[0].color.name
#     return best, max_score
#   else:
#     min_score = float("inf")
#     possible_next_cards = []
#     for card in user_cards:
#       if card.color.name == atut:
#         possible_next_cards.append(card)
    
#     if len(possible_next_cards) > 0:
#       for card in possible_next_cards:
#         new_list = oponent_cards.copy()
#         new_list.remove(card)
#         eval = sec_minimax(user_cards, new_list, atut, True)
#         # Calculate user score?
#         if check_dispatch(card, new_list):
#           score = eval[1] + calculate_dispatch_value(card.color.name)
#         score = eval[1]
#         min_score = min(min_score, score)
#         if min_score == score:
#           best = eval
    
#     else:
#       for card in oponent_cards:
#         new_list = oponent_cards.copy()
#         new_list.remove(card)
#         eval = sec_minimax(user_cards, new_list, atut, True)
#         # Calculate user score?
#         if check_dispatch(card, new_list):
#           score = eval[1] + calculate_dispatch_value(card.color.name)
#         score = eval[1]
#         min_score = min(min_score, score)
#         if min_score == score:
#           best = eval

#     oponent_cards.remove(best[0])
#     if check_dispatch(best[0], oponent_cards):
#       atut = best[0].color.name
#     return best, min_score


In [ ]:
from copy import deepcopy

def check_dispatch(current_card, cards):
  is_dispatch = False
  if current_card.value.name == CardValue.D.name:
    color = current_card.color.name
    for card in cards:
      if card.color.name == color and card.value.name == CardValue.K.D:
        is_dispatch = True
        break
  return is_dispatch

def calculate_dispatch_value(color):
  if color == "PIK":
    return 40
  elif color == "TREFL":
    return 60
  elif color == "KARO":
    return 80
  elif color == "KIER":
    return 100
  return 0

# Z tego co rozumiem to minimax ma minimalizować zysk gracza B, ale zwracamy kartę, którą powinien użyć gracz A

def minmax(user_cards, # A player cards 
          oponent_cards, # B player cards
          atut, # Required color (used to decrease number of options in decision tree)
          oponent_score,
          maximizing_player):
  
  min_score = float("inf")
  if maximizing_player:
    # Wcześniejszą turę wygrał gracz A i on jako pierwszy wykłada kartę
    for user_card in user_cards:
      for oponent_card in oponent_cards:
        score = 0
        is_oponent_win = False

        # Przygotowanie kart do przekazania rekursywnie
        new_user_cards = []
        new_oponent_cards = []
        for elem in user_cards:
          if elem != user_card:
            new_user_cards.append(deepcopy(elem))
        for elem in oponent_cards:
          if elem != oponent_card:
            new_oponent_cards.append(deepcopy(elem))
        
        # Sprawdzenie czy gracz B wygrał obecną ture
        if atut != None and user_card.color.value != atut and oponent_card.color.value == atut:
          score = user_card.value.value + oponent_card.value.value
          is_oponent_win = True
        elif atut != None and oponent_card.color.value == atut and user_card.color.value == atut and oponent_card.value.value > user_card.value.value:
          score = user_card.value.value + oponent_card.value.value
          is_oponent_win = True
        elif atut == None and oponent_card.value.value > user_card.value.value:
          score = user_card.value.value + oponent_card.value.value
          is_oponent_win = True
        
        # Sprawdzenie, czy gracz B może ogłosić meldunek w obecnej turze
        if check_dispatch(oponent_card, new_oponent_cards):
          atut = oponent_card.color.name
          score = score + calculate_dispatch_value(atut)
        
        # Sprawdzenie, czy gracz A może ogłosić meldunek w obecnej turze (możliwa zmiana koloru atutu)
        if check_dispatch(user_card, new_user_cards):
          atut = user_card.color.name
        
        oponent_score = oponent_score + score
        # Wywołanie minimax zgodnie z wynikiem obecnej tury
        if is_oponent_win:
          eval = minmax(new_user_cards, new_oponent_cards, atut, oponent_score, False)
        else:
          eval = minmax(new_user_cards, new_oponent_cards, atut, oponent_score, True)

        # W sumie od tego momentu jest pewien problem, bo nie wiem co zwracać jak rozpatrywana jest gra całymi turami. Wstępnie proponuję zwracanie karty i punktów zdobytych w obecnej turze
        # przez gracza B
        # Odejmowanie punktów zdobytych przez przeciwnika w hipotetycznej kolejnej turze (powrót w górę drzewa)
        oponent_score = oponent_score - eval[1]
        
        return best_card, score
  else:
    # Wcześniejszą turę wygrał gracz B i on jako pierwszy wykłada kartę
    for oponent_card in oponent_cards:
      for user_card in user_cards:
        score = 0
        is_oponent_win = False

        # Przygotowanie kart do przekazania rekursywnie
        new_user_cards = []
        new_oponent_cards = []
        for elem in user_cards:
          if elem != user_card:
            new_user_cards.append(deepcopy(elem))
        for elem in oponent_cards:
          if elem != oponent_card:
            new_oponent_cards.append(deepcopy(elem))
        
        # Sprawdzenie czy gracz B wygrał obecną ture
        if atut != None and user_card.color.value != atut and oponent_card.color.value == atut:
          score = user_card.value.value + oponent_card.value.value
          is_oponent_win = True
        elif atut != None and oponent_card.color.value == atut and user_card.color.value == atut and oponent_card.value.value > user_card.value.value:
          score = user_card.value.value + oponent_card.value.value
          is_oponent_win = True
        elif atut == None and oponent_card.value.value > user_card.value.value:
          score = user_card.value.value + oponent_card.value.value
          is_oponent_win = True
        elif atut == None and oponent_card.value.value == user_card.value.value:
          score = user_card.value.value + oponent_card.value.value
          is_oponent_win = True
        
        # Sprawdzenie, czy gracz B może ogłosić meldunek w obecnej turze
        if check_dispatch(oponent_card, new_oponent_cards):
          atut = oponent_card.color.name
          score = score + calculate_dispatch_value(atut)
        
        # Sprawdzenie, czy gracz A może ogłosić meldunek w obecnej turze (możliwa zmiana koloru atutu)
        if check_dispatch(user_card, new_user_cards):
          atut = user_card.color.name
        
        oponent_score = oponent_score + score
        # Wywołanie minimax zgodnie z wynikiem obecnej tury
        if is_oponent_win:
          eval = minmax(new_user_cards, new_oponent_cards, atut, oponent_score, False)
        else:
          eval = minmax(new_user_cards, new_oponent_cards, atut, oponent_score, True)

        # W sumie od tego momentu jest pewien problem, bo nie wiem co zwracać jak rozpatrywana jest gra całymi turami. Wstępnie proponuję zwracanie karty gracza A i 
        # punktów zdobytych w obecnej turze przez gracza B
        # Odejmowanie punktów zdobytych przez przeciwnika w hipotetycznej kolejnej turze (powrót w górę drzewa)
        oponent_score = oponent_score - eval[1]
        
        return best_card, score